# The relationship between the Human Development Index and the prevalance of mental health disorders

## Research question

### Data sources

### Data loading

### Data inspection and exploration

### Data wrangling

### Normalization

Both data sets are normalized

### Statistical analysis

### Data visualisation